Read drive table from amazon.db

In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


db = sqlite3.connect('../data/amazon.db')

drive_df = pd.read_sql('select * from drive', db)
drive_df.head()

Could not connect to 127.0.0.1: 52562
Traceback (most recent call last):
  File "C:\Program Files\JetBrains\PyCharm 2020.1\plugins\python\helpers\pydev\_pydevd_bundle\pydevd_comm.py", line 458, in start_client
    s.connect((host, port))
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。
Traceback (most recent call last):
  File "C:\Program Files\JetBrains\PyCharm 2020.1\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_utils.py", line 81, in attach_to_debugger
    debugger.connect(pydev_localhost.get_localhost(), debugger_port)
  File "C:\Program Files\JetBrains\PyCharm 2020.1\plugins\python\helpers\pydev\pydevd.py", line 655, in connect
    s = start_client(host, port)
  File "C:\Program Files\JetBrains\PyCharm 2020.1\plugins\python\helpers\pydev\_pydevd_bundle\pydevd_comm.py", line 458, in start_client
    s.connect((host, port))
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。
Failed to connect to target debugger.


,asin,catagory,manufacturer,brand,model_name,product_name,drive_type,drive_subtype,capacity,last_update
0,B01K8A29BE,drive,ADATA USA,ADATA,su800,ADATA SU800 128GB 3D-NAND 2.5 Inch SATA III Hi...,ssd,sata,128gb,5617048
1,B01K8A2A0E,drive,ADATA USA,ADATA,su800,ADATA SU800 256GB 3D-NAND 2.5 Inch SATA III Hi...,ssd,sata,512gb,5617042
2,B01K8A29E6,drive,ADATA USA,ADATA,su800,ADATA USA Ultimate Su800 1TB 3D Nand 2.5 Inch ...,ssd,sata,1tb,5617030
3,B07GBRG2G8,drive,ADATA,ADATA,su800,ADATA SU800 2TB 3D-NAND 2.5 Inch SATA III High...,ssd,sata,2tb,5617030
4,B07TDW8Z3M,drive,ADATA,ADATA,su760,ADATA SU760 256GB 3D NAND 2.5 Inch SATA III In...,ssd,sata,256gb,5617066


Query all time series

In [8]:
def search(asin):
    drive_ts = pd.read_sql_query(f'select keepa_minute, value1 from drive_ts where asin = "{asin}" and keepa_idx=1', db)
    drive_ts['ts'] = pd.to_datetime((drive_ts['keepa_minute'] + 21564000) * 60, unit='s')
    drive_ts.set_index('ts', inplace=True)
    drive_ts = drive_ts.resample('1D').ffill()
    drive_ts = drive_ts.resample('M').mean()
    normal_price = 0
    chia_price = 0
    for d in ['2021-01', '2021-02', '2021-07', '2021-08']:
        normal_price += drive_ts.loc[d]['value1'][0] / 4
    for d in ['2021-03', '2021-04', '2021-05', '2021-06']:
        chia_price += drive_ts.loc[d]['value1'][0] / 4
    return {'normal': normal_price, 'chia': chia_price}

print(search(drive_df['asin'][1]))

KeyError: '2021-01'

In [7]:
df_dict = []

for idx, row in drive_df.iterrows():
    ts = search(row['asin'])
    ts['brand'] = row['brand']
    ts['product'] = row['model_name']
    ts['drive_type'] = row['drive_type']
    ts['drive_subtype'] = row['drive_subtype']
    ts['capacity'] = row['capacity']
    df_dict.append(ts)


df = pd.DataFrame(df_dict)

KeyError: '2021-07'

In [33]:
df.to_csv('ts.csv', index=False)

Close database connection.

In [34]:
db.close()